In [275]:
# data
import numpy as np
import pandas as pd

restaurants = pd.read_csv("https://raw.githubusercontent.com/xoyeon/PDPTW/main/grubhub/0o100t100s1p100/restaurants.txt", sep = '\t')
orders = pd.read_csv("https://raw.githubusercontent.com/xoyeon/PDPTW/main/grubhub/0o100t100s1p100/orders.txt", sep = '\t')
couriers = pd.read_csv("https://raw.githubusercontent.com/xoyeon/PDPTW/main/grubhub/0o100t100s1p100/couriers.txt", sep = '\t')
rest_orders = pd.merge(restaurants, orders, how='inner', on=['restaurant'])
rest_orders.columns = ['restaurant', 'rx', 'ry', 'order', 'ox', 'oy', 'placement_time', 'ready_time']

courier = []
for i in range(len(rest_orders)):
    c = couriers[(couriers['on_time'] <= rest_orders.iloc[i,6]) & (couriers['off_time'] >= rest_orders.iloc[i,7])]
    courier.append(c)

working_all = []
for i in range(len(rest_orders)):
    for j in range(len(courier[i])):
        time = (abs(rest_orders.iloc[i,1] - courier[i].iloc[j,1]) + abs(rest_orders.iloc[i,2]  - courier[i].iloc[j,2])) / 50  <= rest_orders.iloc[i,7]
    if time == True:
        working_all.append(courier[i])
#     if time == False:
#         working_all.append([])

if working_all == []:
    exit()
else:
    pass

In [276]:
# Genetic AL
from random import random, shuffle, randint
from math import floor, sqrt
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# constants
GENERATION_COUNT = 500 # 한 세대, 가능해들의 집합
POPULATION_COUNT = 100 # 한 generation의 총 인구수
MUTATION_PROBABILITY = 0.9
SATURATION_PERCENTAGE = 0.5

In [277]:
# creating a path object
class Path:
    def __init__(self, sequence):
        self.sequence = sequence
        self.distance = 0 
        self.fitness = 0
    def __repr__(self):
        return "{ " + f"Path: {self.sequence}, Fitness: {self.fitness}" + " }"

In [278]:
# initialization
# Create an initial population.
# This population is usually randomly generated and can be any desired size, from only a few individuals to thousands.
def initialization(path, populationCount):
    population = [path] # 클래스 객체
    for i in range(populationCount - 1):
        newPath = path.sequence[:]  ## == 모든 cities의 list
#         while pathExists(newPath, population):  # True일 때 무한반복 --> 여기를 고쳐
        if path.sequence == newPath:
            shuffle(newPath)
        population.append(Path(newPath))
    return population  

# Returns true if the path exists and false otherwise 셔플 유무
def pathExists(path, population):
    for item in [path]:
        if item.sequence == path: ## 내가 지금 여기 그래서 while True
            return True
    return False

In [279]:
# evaluation
# Each member of the population is then evaluated and we calculate a 'fitness' for that individual.
# The fitness value is calculated by how well it fits with our desired requirements.
# These requirements could be simple, 'faster algorithms are better', or more complex, 'stronger materials are better but they shouldn't be too heavy'

'''def go_or_not(path): # 도착하면 1, 못하면 0
    for i in range(len(working_all)):
        give_me_money = working_all[i]
        if give_me_money == [] :
            return 0
        else:
            return 1'''
"---------- 이 부분을 고치자 --------"

def std(path):
    standard = 0
    for i in range(len(path.sequence)):
        if i == len(path.sequence) -1:
            mean = len(rest_orders) / len(working_all)
            dev = sqrt(((len(working_all[i]) - mean)**2)/len(working_all)) # 배달원마다 (배달건수 - 평균)^2 을 더해서 /len(working_all) 루트
            standard += dev
        else:
            mean = len(rest_orders) / len(working_all)
            dev = sqrt(((len(working_all[i+1]) - mean)**2)/len(working_all))
            standard += dev
    path.std = standard
    return standard
    

def goingwithfood(path):
    withfood = 0
    for i in range(len(path.sequence)):
        if i == len(path.sequence) -1:
            print(path.sequence[i])
            print(working_all[path.sequence[i]])
            print(working_all[i]['x'])
        # 그냥 i : , path.sequence[i]
            get_food = (abs(working_all[i]['x'] - rest_orders.iloc[i,1]) + abs(working_all[i][2] - rest_orders.iloc[i,2]))+ (
                abs(rest_orders.iloc[i,1] - rest_orders.iloc[i,4]) + abs(rest_orders.iloc[i,2] - rest_orders.iloc[i,5]))
            print("hihhih")
            withfood += get_food

    path.get_food = withfood
    return withfood

'''def calculateDistance(path):
    total = 0
    for i in range(len(path.sequence)):
        if i == len(path.sequence) - 1:
            distance = abs(restaurants[path.sequence[i]][0] - orders[path.sequence[i]][0]) + abs(
            restaurants[path.sequence[i]][1] - orders[path.sequence[i]][1])
            total += distance
        else:
            distance = abs(restaurants[path.sequence[i+1]][0] - orders[path.sequence[i+1]][0]) + abs(
            restaurants[path.sequence[i+1]][1] - orders[path.sequence[i+1]][1])
            total += distance  
    path.distance = total
    return total'''

def asap(path):
    return goingwithfood / 50

In [280]:
'''거리를 계산하고 min-max scaling
거리에 따른 시간을 계산하고 min-max scaling

일 분배 : 중복없는 순열
- 중복순열은 서로 구별되는 것을 서로 구별되는 것에 이동/분배시킬 때,
- 중복조합은 서로 구별 안되는 것을 서로 구별되는 것에 이동/분배시킬 때,
배달원에게 일거리를 배분하고 min-max scaling (minimize standard variable / maximize confidence level)
std = numpy.std(data) 
신뢰구간 = mean +- 1.96*std/sqrt(len(courier))'''
# 배달원 간 "배달 건 수 표준편차는 작게, 신뢰도(시간 내에 배달 할 수 있는 건 수)는 높게" 계산

'거리를 계산하고 min-max scaling\n거리에 따른 시간을 계산하고 min-max scaling\n\n일 분배 : 중복없는 순열\n- 중복순열은 서로 구별되는 것을 서로 구별되는 것에 이동/분배시킬 때,\n- 중복조합은 서로 구별 안되는 것을 서로 구별되는 것에 이동/분배시킬 때,\n배달원에게 일거리를 배분하고 min-max scaling (minimize standard variable / maximize confidence level)\nstd = numpy.std(data) \n신뢰구간 = mean +- 1.96*std/sqrt(len(courier))'

In [281]:
def calculateFitness(population):
    fit = stdev = distance = time = 0
    for path in population:
        stdev = std(path)

#         distance = goingwithfood(path)
#         print("me1")
        fit += stdev + distance + time  #std + #confidence
        path.fitness = 1/fit
    for path in population:
        path.fitness /= fit
    return sorted(population, key=lambda x: x.fitness, reverse=True)

In [282]:
# Selection
# We want to be constantly improving our populations overall fitness.
# Selection helps us to do this by discarding the bad designs and only keeping the best individuals in the population.
# There are a few different selection methods but the basic idea is the same, make it more likely that fitter individuals will be selected for our next generation.
def select(population):
    randomNumber = random()
    third = floor(0.3 * len(population))
    randomIndex = randint(0, third)
    if randomNumber <= 0.7:
        return population[randomIndex]
    else:
        return population[randint(third+1, len(population) - 1)]

In [283]:
# Crossover
# During crossover we create new individuals by combining aspects of ourselected individuals.
# from two or more individuals we will create an even 'fitter' offspring which will inherit the best traits from We can think of this as mimicking how sex works in nature.
# The hope is that by combining certain traits each of its parents.
def crossOver(population):   ### --> K-means + Binary 추가하기
    father = select(population)
    mother = select(population)
    while(mother == father):
        mother = select(population)
    startIndex = randint(0,len(mother.sequence) - 2)
    endIndex = randint(startIndex + 1, len(mother.sequence) - 1)
    childSequence = [None] * len(population[0].sequence)
    for i in range(startIndex, endIndex + 1):
        childSequence[i] = mother.sequence[i]
    for i in range(len(childSequence)):
        if childSequence[i] is None:
            for j in range(0, len(childSequence)):
                if father.sequence[j] not in childSequence:
                    childSequence[i] = father.sequence[j]
                    break
    return Path(childSequence)

def crossOverTwoHalfandHalf(population):
    father = select(population)
    mother = select(population)
    while(mother == father):
        mother = select(population)
    mid = len(mother.sequence) // 2
    childSequence = [None] * len(mother.sequence)
    for i in range(mid):
        childSequence[i] = mother.sequence[i]
    for i in range(mid, len(father.sequence)):
        for k in range(len(father.sequence)): 
            if father.sequence[k] not in childSequence:
                childSequence[i] = father.sequence[k]
                break 
    return Path(childSequence)

In [284]:
# Mutation 
# We need to add a little bit randomness into our populations' genetics otherwise every combination of solutions we can create would be in our initial population.
# Mutation typically works by making very small changes at random to an individual’s genome.
def mutation(path):
    firstIndex = randint(0, len(path.sequence) - 1)
    secondIndex = randint(0, len(path.sequence) - 1)
    while secondIndex == firstIndex:
        secondIndex = randint(0, len(path.sequence) - 1)
    probability = random()
    if probability < MUTATION_PROBABILITY:
        temp = path.sequence[firstIndex]
        path.sequence[firstIndex] = path.sequence[secondIndex]
        path.sequence[secondIndex] = temp
    return path

def mutationTwoInsertion(path):
    firstIndex = randint(0, len(path.sequence) - 1)
    secondIndex = randint(0, len(path.sequence) - 1)
    while secondIndex == firstIndex:
        secondIndex = randint(0, len(path.sequence) - 1)
    probability = random()
    if probability < MUTATION_PROBABILITY:
        city = path.sequence[firstIndex]
        path.sequence.remove(path.sequence[firstIndex])
        path.sequence.insert(secondIndex, city)
    return path

In [285]:
# Repeat 
# Now we have our next generation we can start again from step two until we reach a termination condition
def geneticAlgorithm(path, populationCount, generationCount):
    path = Path(path)

    population = initialization(path, populationCount)

    population = calculateFitness(population)

    best = population[0]

    print(f"Generation 1: {best.fitness}, distance: {round(best.distance, 2)}")
    saturation = 0

    for i in range(2, generationCount + 1):
        print(f"Generation {i}: {best.fitness}, distance: {round(best.distance, 2)}")
        newGeneration = []
    
        for _ in range(populationCount):
            child = crossOver(population)
            # child = crossOverTwoHalfandHalf(population)
            newGeneration.append(mutation(child))
            # newGeneration.append(mutationTwoInsertion(child))
        population = calculateFitness(newGeneration)
        if population[0].fitness > best.fitness:
            best = population[0]
            saturation = 0
        else:
            saturation += 1
        if saturation > (SATURATION_PERCENTAGE * GENERATION_COUNT):
            break
    return best

In [286]:
# program entry point 
cities = list(range(50))
# cities = list(range(len(working_all)))
best = geneticAlgorithm(cities, POPULATION_COUNT, GENERATION_COUNT)
# plotData(best)
print(best)

Generation 1: 2.105279007121792e-06, distance: 0
Generation 2: 2.105279007121792e-06, distance: 0
Generation 3: 2.105279007121792e-06, distance: 0
Generation 4: 2.105279007121792e-06, distance: 0
Generation 5: 2.105279007121792e-06, distance: 0
Generation 6: 2.105279007121792e-06, distance: 0
Generation 7: 2.105279007121792e-06, distance: 0
Generation 8: 2.105279007121792e-06, distance: 0
Generation 9: 2.105279007121792e-06, distance: 0
Generation 10: 2.105279007121792e-06, distance: 0
Generation 11: 2.105279007121792e-06, distance: 0
Generation 12: 2.105279007121792e-06, distance: 0
Generation 13: 2.105279007121792e-06, distance: 0
Generation 14: 2.105279007121792e-06, distance: 0
Generation 15: 2.105279007121792e-06, distance: 0
Generation 16: 2.105279007121792e-06, distance: 0
Generation 17: 2.105279007121792e-06, distance: 0
Generation 18: 2.105279007121792e-06, distance: 0
Generation 19: 2.105279007121792e-06, distance: 0
Generation 20: 2.105279007121792e-06, distance: 0
Generatio

Generation 171: 2.105279007121792e-06, distance: 0
Generation 172: 2.105279007121792e-06, distance: 0
Generation 173: 2.105279007121792e-06, distance: 0
Generation 174: 2.105279007121792e-06, distance: 0
Generation 175: 2.105279007121792e-06, distance: 0
Generation 176: 2.105279007121792e-06, distance: 0
Generation 177: 2.105279007121792e-06, distance: 0
Generation 178: 2.105279007121792e-06, distance: 0
Generation 179: 2.105279007121792e-06, distance: 0
Generation 180: 2.105279007121792e-06, distance: 0
Generation 181: 2.105279007121792e-06, distance: 0
Generation 182: 2.105279007121792e-06, distance: 0
Generation 183: 2.105279007121792e-06, distance: 0
Generation 184: 2.105279007121792e-06, distance: 0
Generation 185: 2.105279007121792e-06, distance: 0
Generation 186: 2.105279007121792e-06, distance: 0
Generation 187: 2.105279007121792e-06, distance: 0
Generation 188: 2.105279007121792e-06, distance: 0
Generation 189: 2.105279007121792e-06, distance: 0
Generation 190: 2.1052790071217